# 装饰器和闭包

## 装饰器基础知识

装饰器是一种可调用对象，其参数时另一个函数（被装饰的函数）

装饰器的基本性质：

- 装饰器是一个函数或其他可调用对象
- 装饰器可以把被装饰的函数替换成别的函数
- 装饰器在加载（导入）模块时立即执行。
- 严格来说装饰器只是语法糖。

In [7]:
def deco(func):
    def inner():
        print('running inner()')
    return inner                 # deco 返回内部函数对象inner

@deco
def target():                    # 使用deco装饰target
    print('running target()')   

target()            # 调用被装饰的 target，运行的其实是 inner

running inner()


## 何时执行装饰器

In [8]:
registry = []  # <1>

def register(func):  # <2>
    print(f'running register({func})')  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()  # <9>

running register(<function f1 at 0x000001E19E15A200>)
running register(<function f2 at 0x000001E19E15AE60>)
running main()
registry -> [<function f1 at 0x000001E19E15A200>, <function f2 at 0x000001E19E15AE60>]
running f1()
running f2()
running f3()


## 变量作用域规则

In [10]:
b = 6
def f2(a):
    print(a)
    print(b)

f2(3)

3
6


In [11]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9     # 作用域报错

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

## 闭包

In [16]:
class Averager():    # 类实现

    def __init__(self):
        self.series = []

    def __call__(self,new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/ len(self.series)

avg = Averager()
avg(10)

10.0

In [17]:
avg(11)

10.5

In [19]:
def make_averager():     # 高阶函数 
    series = []                    # 闭包

    def averager(new_value):
        series.append(new_value)   # 自由变量 
        total = sum(series)
        return total / len(series)

    return averager


avg  = make_averager()

avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg.__code__.co_varnames

('new_value', 'total')

In [22]:
avg.__code__.co_freevars

('series',)

In [23]:
avg.__closure__

(<cell at 0x000001E19D7EF5E0: list object at 0x000001E19DC4F300>,)

In [24]:
avg.__closure__[0].cell_contents

[10, 11]

## nonlocal 声明

nonlocal：把变量标记为自由变量，便于在函数中为变量赋予新值。

In [28]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total +=new_value
        return total /count 
    
    return averager


avg = make_averager()
avg(10)

10.0

## 变量查找逻辑

- 如果是global x声明，则x来自模块全局作用域，并赋予那个作用域中x的值。
- 
如果是nonlocal x声明，则x来自最近一个定义它的外层函数，并赋予那个函数中局部变量x的值
- 
如果x是参数，或者在函数主体中赋了值，那么x就是局部变
- 。
如果引用了x，但是没有赋值也不是参数，则需要遵循以下
    - 则：
在外层函数主体的局部作用域（非局部作用域）内
    - 找x。
如果在外层作用域类没有找到，则从模块全局作用
    - 内读取。
如果在模块全局作用域内没有找到，则从__builtins__.__dict
 
## 实现一个简单的装饰器


一个会显示函数运行时间的简单的装饰器__中读取。

In [34]:
import time 

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args) 
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}') 
        return result
    return clocked

@clock
def snnooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


if __name__ == '__main__': 
    print('*' * 40, 'Calling snooze(.123)') 
    snooze(.123) 
    print('*' * 40, 'Calling factorial(6)') 
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12713100s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000030s] factorial(1) -> 1
[0.00000760s] factorial(2) -> 2
[0.00001080s] factorial(3) -> 6
[0.00001360s] factorial(4) -> 24
[0.00001670s] factorial(5) -> 120
[0.00002060s] factorial(6) -> 720
6! = 720


# 工作原理


In [ ]:
@clock 
def factorial(n): 
    return 1 if n < 2 else n*factorial(n-1)

# 其实等价于以下内容

def factorial(n): 
    return 1 if n < 2 else n*factorial(n-1) 

factorial = clock(factorial)

装饰器的主要作用是在每次调用被装饰的函数时计时，把运行时间、传入的参数和调用的结果打印出来。

In [2]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        # 调用被装饰的函数
        result = func(*args, **kwargs)
        
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        
        return result
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


In [31]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[0.12899290s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000030s] factorial(1) -> 1
[0.00000910s] factorial(2) -> 2
[0.00001300s] factorial(3) -> 6
[0.00001620s] factorial(4) -> 24
[0.00001980s] factorial(5) -> 120
[0.00002380s] factorial(6) -> 720
6! = 720


## 标准库中的装饰器

### functools.cache

实现备忘功能，能把耗时的函数得到的结果保存起来，避免传入相同的参数时重复计算。

In [4]:
import functools


@functools.cache
@clock              # 上面clock函数
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


In [5]:
fibonacci(6)

[0.00000070s] fibonacci(0) -> 0
[0.00000140s] fibonacci(1) -> 1
[0.00028440s] fibonacci(2) -> 1
[0.00000130s] fibonacci(3) -> 2
[0.00031440s] fibonacci(4) -> 3
[0.00000100s] fibonacci(5) -> 5
[0.00034200s] fibonacci(6) -> 8


8

In [ ]:
类似于：上述的叠放装饰器相当于fibonacci = functools.cache(clock(fibonacci)

## 单分派泛化函数

`functools.singledispatch` 装饰器可以把整体方案拆分成多个模块，甚至可以为第三方包中无法编辑的类型提供专门的函数，将普通函数变成了泛化函数的入口，即为单分派。如果根据多个参数选择专门的函数，则是多分派。


开发一个调试Web应用程序的工具，生成HTML，以显示不同类型的Python对象。需要满足如下功能：

- 当参数为str时，内部的换行符替换为<br/>\n，不使用<pre>标签，使用<p>。
- 当参数为int时，以十进制和十六进制显示数（bool除外）。
- 当参数为list时，输出一个HTML列表，根据各项的类型进行格式化。
- 当参数为float和Decimal时，正常输出值，外加分数形式。

In [6]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str: 
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'


In [7]:
htmlize({1, 2, 3})


'<pre>{1, 2, 3}</pre>'

In [8]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [9]:
htmlize('Heimlich & Co.\n- a game')


'<p>Heimlich &amp; Co.<br/>\n- a game</p>'

In [10]:
htmlize(42)


'<pre>42 (0x2a)</pre>'

In [11]:
print(htmlize(['alpha', 66, {3, 2, 1}]))


<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [12]:
htmlize(True)


'<pre>True</pre>'

In [13]:
htmlize(decimal.Decimal('0.02380952'))


'<pre>0.02380952 (1/42)</pre>'

## 参数化装饰器

接收其他参数的装饰器：创建一个装饰器工厂函数来接收那些参数，然后再返回一个装饰器，应用到被装饰的函数上。

In [14]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT): # clock是参数化装饰器的工厂函数
    def decorate(func):     # 真正的装饰器 
        def clocked(*_args):  # 包装被装饰的函数 
            t0 = time.perf_counter()
            _result = func(*_args)  # 调用被装饰的函数
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) 
            result = repr(_result)  
            print(fmt.format(**locals()))
            return _result

        return clocked 

    return decorate


In [15]:
@clock()
def snooze(seconds):
    time.sleep(seconds)


for i in range(3):
    snooze(.123)


[0.13265860s] snooze(0.123) -> None
[0.12444340s] snooze(0.123) -> None
[0.12380400s] snooze(0.123) -> None


In [16]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)


snooze: 0.13264110000000073s
snooze: 0.12490290000005189s
snooze: 0.12560629999995854s


In [17]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)


snooze(0.123) dt=0.134s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s


## 基于类的clock装饰器

In [19]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:

    def __init__(self, fmt=DEFAULT_FMT):  
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked
